In [1]:
import pandas as pd

In [2]:
get_data = pd.read_csv("https://bit.ly/seoul-120-text-csv")
get_data

,번호,분류,제목,내용,내용번호
0,2645,복지,아빠 육아휴직 장려금,아빠 육아휴직 장려금 업무개요 남성근로자의 육아휴직을 장려하고 양육에 따른 경...,23522464
1,2644,경제,[서울산업진흥원] 서울메이드란?,서울산업진흥원 서울메이드란 서울의 감성을 담은 다양하고 새로운 경험을 제공하기 위해...,23194045
2,2643,환경,(강북구) 정비중,강북구 정비중 업무개요 투명 폐트병을 교환보상하므로 수거율을 높이고 폐기물을 감...,23032485
3,2642,복지,"광진맘택시 운영(임산부,영아 양육가정 전용 택시)",광진맘택시 운영임산부영아 양육가정 전용 택시 업무개요 교통약자인 임산부와 영아가정...,22904492
4,2641,복지,마포 뇌병변장애인 비전센터,마포 뇌병변장애인 비전센터 마포뇌병변장애인 비전센터 운영 구분 내용 목적 학...,22477798
...,...,...,...,...,...
2640,5,주택도시계획,이미 지정된 재정비촉진지구의 인접지를 추가 확장하고 싶은데 어떻게 해야 하나요?,이미 지정된 재정비촉진지구의 인접지를 추가 확장하고 싶은데 어떻게 해야 하나요 관할...,2894353
2641,4,환경,수돗물의 톤당 원가는?,수돗물의 톤당 원가는우리시 년 톤당 원가는 원 입니다 우리시 년 톤당 요금은 원 입니다,19353862
2642,3,복지,출산휴가후 집근처에 안심하고 맡길 수 있는 어린이집이 있는지?,출산휴가후 집근처에 안심하고 맡길 수 있는 어린이집이 있는지 서울시 육아종합지원...,19699584
2643,2,교통,자동차검사장을 지정받고자 하는데 어떻게 해야 하나요?,자동차검사장을 지정받고자 하는데 어떻게 해야 하나요 자동차종합정비업체 소형자동차정...,2898293


In [3]:
# 불용어 리스트 생성 (예시)
stopwords = ['가', '고', '을', '를', '이', '는']
from konlpy.tag import Okt

# Okt 형태소 분석기 인스턴스 생성
okt = Okt()

# 토크나이징 함수 정의
def tokenizer(raw, pos=["Noun","Alpha","Verb","Number"], stopword=stopwords):
    return [
        word for word, tag in okt.pos(
            raw, 
            norm=True,   # normalize 그랰ㅋㅏ -> 그래ㅋㅋ
            stem=True    # stemming 바뀌나->바뀌다
            )
            if len(word) > 1 and tag in pos and word not in stopword

    ]

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(tokenizer=tokenizer, use_idf=True)
features = vectorizer.fit_transform(get_data['내용'])
features.toarray()

/usr/local/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [5]:
from sklearn.decomposition import LatentDirichletAllocation
lda_model = LatentDirichletAllocation(n_components=5, random_state=55) #인스턴스화 #n_components 토픽의 갯수
lda_model.fit(features) #교육

LatentDirichletAllocation(n_components=5, random_state=55)

In [6]:
dictionary_list = vectorizer.get_feature_names_out()

In [7]:
lda_model.components_

array([[0.26052268, 0.20001471, 0.3200244 , ..., 0.20011546, 0.20003807,
        0.20003309],
       [0.53396432, 0.69258791, 0.20002079, ..., 0.20011499, 0.21341029,
        0.20452152],
       [0.20012092, 0.20002475, 0.20003709, ..., 0.20019725, 0.43243173,
        0.20004293],
       [0.36756002, 0.20002416, 0.20003146, ..., 0.20563783, 0.2000553 ,
        0.45016806],
       [0.30033162, 0.20001479, 0.21010429, ..., 0.50771103, 0.23676219,
        0.24141915]])

In [8]:
topics_output = lda_model.transform(features)
df_topics_score = pd.DataFrame(data=topics_output)

In [9]:
import numpy as np

In [10]:
df_topics_score['dominant_topic_number'] = np.argmax(topics_output, axis=1)

In [11]:
df_topics_score['sentences'] = get_data['내용']

In [12]:
df_topics_score

,0,1,2,3,4,dominant_topic_number,sentences
0,0.161565,0.028160,0.026227,0.026247,0.757801,4,아빠 육아휴직 장려금 업무개요 남성근로자의 육아휴직을 장려하고 양육에 따른 경...
1,0.039284,0.037992,0.407289,0.038134,0.477301,4,서울산업진흥원 서울메이드란 서울의 감성을 담은 다양하고 새로운 경험을 제공하기 위해...
2,0.673380,0.031783,0.028916,0.029143,0.236777,0,강북구 정비중 업무개요 투명 폐트병을 교환보상하므로 수거율을 높이고 폐기물을 감...
3,0.026072,0.025931,0.235598,0.025180,0.687219,4,광진맘택시 운영임산부영아 양육가정 전용 택시 업무개요 교통약자인 임산부와 영아가정...
4,0.039994,0.031100,0.030668,0.104691,0.793548,4,마포 뇌병변장애인 비전센터 마포뇌병변장애인 비전센터 운영 구분 내용 목적 학...
...,...,...,...,...,...,...,...
2640,0.335021,0.445529,0.033061,0.152660,0.033729,1,이미 지정된 재정비촉진지구의 인접지를 추가 확장하고 싶은데 어떻게 해야 하나요 관할...
2641,0.072702,0.070849,0.070315,0.712236,0.073897,3,수돗물의 톤당 원가는우리시 년 톤당 원가는 원 입니다 우리시 년 톤당 요금은 원 입니다
2642,0.036857,0.036736,0.036772,0.326996,0.562639,4,출산휴가후 집근처에 안심하고 맡길 수 있는 어린이집이 있는지 서울시 육아종합지원...
2643,0.048133,0.813744,0.045838,0.046061,0.046225,1,자동차검사장을 지정받고자 하는데 어떻게 해야 하나요 자동차종합정비업체 소형자동차정...


# 토픽 주제 확인

In [30]:
## 상위 단어 추출
## 0 확률 1은 dictionary
topics_list = list()
for topic in lda_model.components_:
    df_datas = [topic, dictionary_list]
    df_topics = pd.DataFrame(data=df_datas)
    df_topics= df_topics.T
    df_topics = df_topics.sort_values(0, ascending=False)
    # print(df_topics[:3])
    topics_text = ' '.join(df_topics[1].values[:10])# 시리즈 형식으로 출력 get values from series / index
    print(topics_text)
    topics_list.append(topics_text)


topics_list_add = [topics_list, ['Topic0', 'Topic1', 'Topic2','Topic3','Topic4']]
df_topics_keywords = pd.DataFrame(topics_list_add)

돋다 하다 되다 사업 경우 신고 발급 이상 확인 등록
하다 계획 주택 되다 경우 사업 시설 건축물 관리 도로
택시 하다 에너지 위원장 초등학교 이용 측량 용적률 운수 길음
하다 재정 촉진 지역 경관 도로 설치 도시 서대문구 지구
하다 교육 센터 민원 운영 공무원 되다 사업 지원 신청


In [31]:
df_topics_keywords

,0,1,2,3,4
0,돋다 하다 되다 사업 경우 신고 발급 이상 확인 등록,하다 계획 주택 되다 경우 사업 시설 건축물 관리 도로,택시 하다 에너지 위원장 초등학교 이용 측량 용적률 운수 길음,하다 재정 촉진 지역 경관 도로 설치 도시 서대문구 지구,하다 교육 센터 민원 운영 공무원 되다 사업 지원 신청
1,Topic0,Topic1,Topic2,Topic3,Topic4


- 0번 그룹은 사업에대한 신고,발급,등록 등에 대한 민원으로 보여진다.
- 1번 그룹은 주택,시설,건축물에 대한 사업,도로 등의 관리민원으로 보여진다.
- 2번 그룹은 학교와 운수에 대한 민원이 섞여있는 것으로 보여진다.
- 3번 그룹은 도시 내 지역 재정 촉진, 도로, 경관 등의 지역 시설 민원으로 보여진다.
- 4번 그룹은 민간에 대한 교육, 지원 센터, 사업 등의 국민 지원제도에 대한 민원으로 보여진다.

## 2번 그룹이 섞여있는 것으로 보여지므로 추출하여 확인

In [33]:
df_second = df_topics_score.query('dominant_topic_number == 2')
df_second

,0,1,2,3,4,dominant_topic_number,sentences
9,0.332096,0.025827,0.361378,0.024098,0.256600,2,반반택시 심야시간 택시 동승앱 업무개요 심야 택시수요 대비 택시공급 부족으로 발...
11,0.035667,0.035784,0.490857,0.034851,0.402840,2,종료사업 앱 에스택시 서울택시승차앱 시범운영업무개요 빈차를 보고 택시를 이용 하는 ...
15,0.266253,0.030360,0.517361,0.029797,0.156229,2,자동배차콜택시웨이고 블루여성전용예약콜택시웨이고 레이디 구분 내 용 업...
19,0.052699,0.418958,0.435041,0.045977,0.047324,2,사업이란 뭔가요란 에너지절약전문기업의 약자로 에너지사용자가 에너지절약을 위하여 기...
123,0.094119,0.020054,0.474473,0.020277,0.391077,2,지하철 스탬프 투어 업무개요 서울메트로와 서울도시철도공사에서 진행하는 프...
...,...,...,...,...,...,...,...
2581,0.057419,0.057983,0.772287,0.055611,0.056699,2,승차 또는 하차단말기가 고장이 난 경우는 앞문 및 뒷문 단말기는 승차와 하차가 가능...
2582,0.045380,0.333231,0.526538,0.045024,0.049828,2,시립대학교 외부인 출입 학교 출입은 인근 주민들의 편의를 위하여 산책 등에 한해 제...
2599,0.036227,0.168047,0.723529,0.035869,0.036327,2,전기분해실험에서 적갈색 앙금이 수돗물에서 생기던데요 수돗물에서 발생하는 적갈색 앙금...
2623,0.040524,0.397550,0.481871,0.039677,0.040379,2,광암정수장 여과지에 대해알고 싶습니다ㅁ여과지침전물이나 불순물을 걸러내기 위해 사용하...


In [52]:
counter = 0
check_list = []
for i in df_second['sentences']:
    token = tokenizer(i)
    temp_dic = {}
    for j in token:
        if temp_dic.get(j,True): # 민원 하나에 대한 내용 복잡도를 줄이기 위해 중복되는 단어 제거
            temp_dic[j] = temp_dic.get(j, False)
            check_list.append(j)

In [53]:
from collections import Counter
count_dic = Counter(check_list)

In [54]:
list(count_dic)[:10]

['택시', '심야시간', '동승', '업무', '개요', '심야', '수요', '대비', '공급', '부족']

- 단어의 수를 확인한 결과 심야시간에 대한 공공이동 수단의 민원이 주를 이루는 것으로 보여진다.